In [16]:
from data_utils import TgsDataset
from rle import mask2rle
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import torch
from skimage import io, transform

In [2]:
batch_size = 40
test_df = pd.read_csv('tgs/sample_submission.csv')
tgs_test_dataset = TgsDataset(test_df,'tgs/test/images')
tgs_test_loader = DataLoader(tgs_test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [3]:
model = torch.load('tgs_11_23_15_11.pth')

In [4]:
device = torch.device('cuda')
dtype = torch.float32

In [7]:
masks = []
for i, sample_batch in enumerate(tgs_test_loader):
    images = sample_batch['image'].to(device=device, dtype=dtype) / 1000000
    outputs = model(images)
    masks.append(outputs.to(device=torch.device('cpu')).detach().numpy())
masks = np.concatenate(masks)

(18000, 1, 128, 128)


In [18]:
for i in range(18000):
    mask = transform.resize(masks[i].reshape(128,128),(101,101))
    mask_zero = np.zeros_like(mask)
    mask_zero[mask>0.5] = 1
    test_df.iloc[i,1] = mask2rle(mask_zero)

In [20]:
test_df.to_csv('submission.csv',index=False)